In [1]:
#!pip install colabcode
# !pip install fastapi
!pip install flask-ngrok
!pip install flask

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import os
os.chdir('/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server')

In [4]:
from flask import Flask, jsonify
import pickle
import os
import cv2
import numpy as np 
import datetime
import argparse
import os
import re
import random
import shutil
import time
import numpy as np
import cv2
import json
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow 
from torchvision import transforms
from flask import Flask, url_for, redirect, render_template, request
from flask_ngrok import run_with_ngrok

from data.data_reader import DatasetReader
from data.data_splitter import DatasetSplit
from data.data_transformer import DatasetTransform
from data.transforms import SelectFrames, FrameDifference, Downsample, TileVideo, RandomCrop, Resize, RandomHorizontalFlip, Normalize, ToTensor

In [5]:
model_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server/model/model_best.E_bi_max_pool_ALL_fold_0_t2021-08-12 20:02:12.tar'

def validate(val_loader, model):
    model.eval()

    end = time.time()
    print(type(val_loader))
    input = None
    for data in val_loader:
        input = data
        break
    input_var = torch.autograd.Variable(input, volatile=True)
    output_dict = model(input_var)    

    return output_dict['classification']

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target):
    """Computes the precision of model"""
    batch_size = target.size(0)

    _, pred = torch.max(output.data, 1)

    correct = pred.eq(target).cpu().sum()

    res = correct * 100.0 / batch_size
    return res

def fold(folds, data):                      
    for i in range(folds):                
        train_dataset = None
        val_dataset = DatasetSplit(data)
        yield (train_dataset, val_dataset)

def network_factory(arch):

    if arch == 'E':
        from networks.E import VP

    elif arch == 'E_bi':
        from networks.E_bi import VP

    elif arch == 'E_bi_avg_pool':
        from networks.E_bi_avg_pool import VP

    elif arch == 'E_bi_max_pool':
        from networks.E_bi_max_pool import VP

    else:
        assert 0, "Bad network arch name: " + arch

    return VP

def read_video(filename):
    frames = []
    #print(filename)
    if not os.path.isfile(filename):
        print('file not found')
    cap = cv2.VideoCapture(filename)
    while cap.isOpened():
       ret, frame = cap.read()  
       if not ret:
           break
       frames.append(frame)
    cap.release()
    video = np.stack(frames)
    #print("printing vidoe shape : ", video.shape)
    return video

In [6]:
seed = 250
args = {
    'arch':'E_bi_max_pool',
    'workers':1,
    'batch-size':1,
    'evalmodel':model_path,
    'kfold':0,
    'split':5,
    'frames':5,
    'lr':1e-6,
    'weight_decay':1e-1
}

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

# create model
print("=> creating model '{}'".format(args['arch']))
VP = network_factory(args['arch'])

model = VP()
model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), args['lr'],
                                weight_decay=args['weight_decay'])

# optionally resume from a checkpoint
if os.path.isfile(args['evalmodel']):
    print("=> loading checkpoint '{}'".format(args['evalmodel']))
    checkpoint = torch.load(args['evalmodel'])
    args['start_epoch'] = checkpoint['epoch']
    print('start_epoch : ', args['start_epoch'])
    best_prec = checkpoint['best_prec']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})"
            .format(args['evalmodel'], checkpoint['epoch']))
else:
    print("=> no checkpoint found at '{}'".format(args['evalmodel']))

=> creating model 'E_bi_max_pool'
=> loading checkpoint '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server/model/model_best.E_bi_max_pool_ALL_fold_0_t2021-08-12 20:02:12.tar'
start_epoch :  3
=> loaded checkpoint '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server/model/model_best.E_bi_max_pool_ALL_fold_0_t2021-08-12 20:02:12.tar' (epoch 3)


In [13]:
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route('/')
def index():
    return 'nice'  

violence_files = []
save_video_path = '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Server/saved video/'

@app.route('/predict', methods=['POST'])
def prediction():
    if request.method == 'POST':
        end = time.time()

        file = request.files['file']
        frames = file.read()
        
        FILE_OUTPUT = str(time.strftime('%c', time.localtime(time.time())))+'.mp4'
        if os.path.isfile(FILE_OUTPUT):
            os.remove(FILE_OUTPUT)

        out_file = open(save_video_path + FILE_OUTPUT, "wb") 
        out_file.write(frames)
        out_file.close()

        video_path = save_video_path + FILE_OUTPUT
        
        video = read_video(video_path)
        val_dataset = DatasetReader(video)
        val_transformations = transforms.Compose([Resize(size=224), SelectFrames(num_frames=args['frames']), FrameDifference(dim=0), Normalize(), ToTensor()])

        val_dataset = DatasetTransform(val_dataset, val_transformations)
        val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=1, pin_memory=False)

        model.eval()

        for i, (input) in enumerate(val_loader):
            input_var = torch.autograd.Variable(input, volatile=True)
            input_var = input_var.cuda()

            # compute output
            output_dict = model(input_var)
            output = output_dict['classification']

            # 0,1 classification
            #_, pred = torch.max(output.data, 1)
            #model_ret = pred.cpu().detach().numpy()[0]
            #print('violence? : ',model_ret, ' / ',output.data)

            # softmax layer added
            model_ret = output_dict['classification'].cpu().detach().numpy()[0][1] * 100
            print('violence : ', model_ret,' %, - ',FILE_OUTPUT)
            if model_ret >50:
                print('-----------------violence detected!!!!!!----------------')
                print(model_ret, '% violence detected')
                violence_files.append(FILE_OUTPUT)
                
            #if model_ret == 1:
            #    print('-----------------violence detected!!!!!!----------------')

        print('model calulation time : ',round(time.time() - end,3), ' sec')

        return json.dumps(str(model_ret * 100))

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9073e58e6486.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
127.0.0.1 - - [17/Aug/2021 01:59:30] "POST /predict HTTP/1.1" 200 -


violence :  9.980575740337372  %, -  Tue Aug 17 01:59:30 2021.mp4
model calulation time :  0.625  sec


...

violence :  89.56581354141235  %, -  Tue Aug 17 01:59:54 2021.mp4
-----------------violence detected!!!!!!----------------
89.56581354141235 % violence detected
model calulation time :  4.428  sec


127.0.0.1 - - [17/Aug/2021 01:59:58] "POST /predict HTTP/1.1" 200 -


violence :  84.89543795585632  %, -  Tue Aug 17 01:59:58 2021.mp4
-----------------violence detected!!!!!!----------------
84.89543795585632 % violence detected
model calulation time :  3.582  sec


127.0.0.1 - - [17/Aug/2021 02:00:00] "POST /predict HTTP/1.1" 200 -


violence :  75.18370151519775  %, -  Tue Aug 17 01:59:59 2021.mp4
-----------------violence detected!!!!!!----------------
75.18370151519775 % violence detected
model calulation time :  3.279  sec


In [12]:
    violence_files

['Tue Aug 17 01:53:09 2021.mp4',
 'Tue Aug 17 01:53:18 2021.mp4',
 'Tue Aug 17 01:53:32 2021.mp4',
 'Tue Aug 17 01:53:44 2021.mp4',
 'Tue Aug 17 01:53:50 2021.mp4',
 'Tue Aug 17 01:53:50 2021.mp4',
 'Tue Aug 17 01:53:53 2021.mp4',
 'Tue Aug 17 01:53:54 2021.mp4',
 'Tue Aug 17 01:53:54 2021.mp4',
 'Tue Aug 17 01:54:01 2021.mp4',
 'Tue Aug 17 01:54:02 2021.mp4',
 'Tue Aug 17 01:54:03 2021.mp4',
 'Tue Aug 17 01:54:10 2021.mp4',
 'Tue Aug 17 01:54:14 2021.mp4',
 'Tue Aug 17 01:54:15 2021.mp4',
 'Tue Aug 17 01:54:18 2021.mp4',
 'Tue Aug 17 01:54:23 2021.mp4',
 'Tue Aug 17 01:54:25 2021.mp4']